<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Var" data-toc-modified-id="Var-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Var

In [3]:
work_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n9000-rand_r10/train/'


# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(LeyLabRMisc)

In [5]:
df.dims()
setDTthreads(4)

# Load

In [6]:
D = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n9000-rand_r10/train/features/'
F = file.path(D, 'feature_files.tsv')
files = Fread(F) %>%
    mutate.(feature_file = file.path(D, feature_file))
files

richness,rep,read_depth,assembler,feature_file,MGSIM_reads_params
<dbl>,<int>,<int>,<chr>,<chr>,<chr>
0.055555,1,500000,metaspades,/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n9000-rand_r10/train/features//0.055555/1/500000/metaspades/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10
0.055555,1,500000,megahit,/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n9000-rand_r10/train/features//0.055555/1/500000/megahit/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10
⋮,⋮,⋮,⋮,⋮,⋮
0.333333,10,4000000,metaspades,/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n9000-rand_r10/train/features//0.333333/10/4000000/metaspades/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10
0.333333,10,4000000,megahit,/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/deepmased-sm/n9000-rand_r10/train/features//0.333333/10/4000000/megahit/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10


In [12]:
read_feat = function(F){
    Fread(F) %>%
        select.(contig, coverage, num_SNPs, mean_insert_size_Match, mean_insert_size_SNP) %>%
        summarize.(coverage = median(coverage, na.rm=TRUE),
                   num_SNPs = median(num_SNPs, na.rm=TRUE),
                   mean_insert_size_Match = median(mean_insert_size_Match, na.rm=TRUE),
                   sd_insert_size_Match = sd(mean_insert_size_Match, na.rm=TRUE),
                   mean_insert_size_SNP = median(mean_insert_size_SNP, na.rm=TRUE),
                   sd_insert_size_SNP = sd(mean_insert_size_SNP, na.rm=TRUE),
                   .by=c(contig))
}
doParallel::registerDoParallel(8)
df = files$feature_file %>%
    plyr::llply(read_feat, .parallel=TRUE) %>%
    rbindlist(use.names=TRUE) 
df

contig,coverage,num_SNPs,mean_insert_size_Match,sd_insert_size_Match,mean_insert_size_SNP,sd_insert_size_SNP
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NODE_1169_length_1013_cov_2.305846,4,0,253.30,6.181070,252,8.485281
NODE_890_length_1158_cov_1.858568,3,0,249.85,4.270082,256,6.898067
⋮,⋮,⋮,⋮,⋮,⋮,⋮
k141_14989,2,0,246,4.45962,248,4.242641
k141_87444,5,0,250,34.57184,243,2.566718


In [13]:
# per-contig coverage distribution
summary_x(df$coverage)
quantile(df$coverage, 0.95)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
coverage,1,2,3,3.577674,4,1553,632.902,258.381


95% 
  6

In [14]:
# insert sizes
summary_x(df$mean_insert_size_Match)
summary_x(df$mean_insert_size_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
mean_insert_size_Match,89.1,247.8,249.5,249.4019,251.25,288.6,70.44,28.757


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_insert_size_SNP,136,246,250,249.7776,254,496,195315,73719.42,27863.32


In [15]:
# match/SNP
summary_x(df$sd_insert_size_Match)
summary_x(df$sd_insert_size_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sd_insert_size_Match,0.6343207,5.218077,6.69007,13.62339,10.24393,3489.589,1421.653,580.387


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
sd_insert_size_SNP,0,5.152375,7.931162,8.0811,10.53722,141.5307,634824,239930,90685.02


In [16]:
# SNPs
summary_x(df$num_SNPs)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_SNPs,0,0,0,2.463306e-06,0,1,0.408,0.167


In [17]:
send_email('http://127.0.0.1:9704/notebooks/deepmased/notebooks/06_version2/01_simulation_datasets/00_dev/00_n9000-r10_train_stats.ipynb')

Sending email to: nyoungblut@tuebingen.mpg.de



# sessionInfo

In [19]:
sessionInfo()

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Anxiety_Twins_Metagenomes/envs/tidyverse2/lib/libopenblasp-r0.3.12.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.9 tidytable_0.6.3   data.table_1.14.0 ggplot2_3.3.5    
[5] tidyr_1.1.3       dplyr_1.0.7      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.7        plyr_1.8.6        pillar_1.6.1      compiler_4.1.0   
 [5] iterators_1.0.13  R.methodsS3_1.8.1 